In [1]:
# импортируем
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# считываем файлы casc-resto и CASC_Constant в переменные orders и clients соответственно
# разделитель в первом случае - ';', для отделения десятичной доли в файле используется запятая, поэтому не забудем это
orders = pd.read_csv('casc-resto.csv', sep=';', decimal=',')
clients = pd.read_csv('CASC_Constant.csv')

In [3]:
# для проверки посмотрим первые несколько записей, вроде бы все ок
orders.head(20)

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.0,2.00
1,2903215,43,2015-04-07,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
2,2748887,43,2015-05-22,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
3,2862077,46,2015-03-05,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
4,2862077,46,2015-03-11,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
5,2862077,46,2015-03-13,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
6,2862077,46,2015-04-20,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
7,2862077,46,2015-05-13,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
8,2801997,46,2015-05-20,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.00
9,2862077,46,2015-05-28,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,3.21


In [4]:
# дату надо конвертировать в другой тип (RKDate)
orders.dtypes

CustomerID                int64
Restaurant                int64
RKDate                   object
RegionName               object
BrandsNames              object
DishCategoryName         object
Quantity                  int64
SummBasic               float64
SummAfterPointsUsage    float64
dtype: object

In [5]:
clients.head(20)

,CustomerId,ActivationDate,Age,Sex,SubscribedEmail,SubscribedPush
0,2728183,2015-01-01,21.0,Female,False,True
1,2728198,2015-01-01,21.0,Female,True,True
2,2728306,2015-01-01,21.0,Female,True,True
3,2728178,2015-01-01,22.0,Male,True,True
4,2728322,2015-01-01,22.0,Male,True,True
5,2728319,2015-01-01,22.0,NaN,False,False
6,2728481,2015-01-01,23.0,Female,True,False
7,2728244,2015-01-01,23.0,Female,True,True
8,2728524,2015-01-01,23.0,Male,False,False
9,2728514,2015-01-01,24.0,Female,True,True


In [6]:
# то же самое с ActivationDate
clients.dtypes

CustomerId           int64
ActivationDate      object
Age                float64
Sex                 object
SubscribedEmail       bool
SubscribedPush        bool
dtype: object

In [7]:
# смотрим статистику по заказам. В глаза бросается тот факт, что после скидки (вероятно) стоимость заказа отрицательная
orders.describe()

,CustomerID,Restaurant,Quantity,SummBasic,SummAfterPointsUsage
count,8.822220e+05,882222.000000,882222.000000,882222.000000,882222.000000
mean,2.809199e+06,468.268804,1.154900,287.725795,260.255589
std,6.178420e+04,279.170967,0.650287,264.555291,251.289517
min,2.728046e+06,40.000000,0.000000,0.500000,-2593.000000
25%,2.754886e+06,333.000000,1.000000,120.000000,110.000000
50%,2.785104e+06,434.000000,1.000000,225.000000,199.000000
75%,2.878900e+06,712.000000,1.000000,379.000000,348.940000
max,2.913132e+06,980.000000,107.000000,29450.000000,29450.000000


In [8]:
# смотрим статистику по клиентам. В глаза бросается тот факт, что максимальный возраст - 247, нереалистично
clients.describe()

,CustomerId,Age
count,1.000000e+04,9953.000000
mean,2.812482e+06,32.894203
std,6.227632e+04,11.954687
min,2.728046e+06,16.000000
25%,2.756356e+06,26.000000
50%,2.794998e+06,31.000000
75%,2.880618e+06,37.000000
max,2.913132e+06,247.000000


In [9]:
# меняем тип у даты
orders['RKDate'] = pd.to_datetime(orders['RKDate'])
clients['ActivationDate'] = pd.to_datetime(clients['ActivationDate'])

# удаляем строки с пустыми значениями из таблицы
orders = orders.dropna()
clients = clients.dropna()

In [10]:
# удалили 882222-881608 значений, well done
orders.describe()

,CustomerID,Restaurant,Quantity,SummBasic,SummAfterPointsUsage
count,8.816080e+05,881608.000000,881608.000000,881608.000000,881608.000000
mean,2.809192e+06,467.963099,1.154983,287.675306,260.209665
std,6.178279e+04,279.025459,0.650488,264.618404,251.348268
min,2.728046e+06,40.000000,0.000000,0.500000,-2593.000000
25%,2.754872e+06,325.000000,1.000000,120.000000,110.000000
50%,2.785104e+06,434.000000,1.000000,225.000000,199.000000
75%,2.878847e+06,712.000000,1.000000,379.000000,348.832500
max,2.913132e+06,980.000000,107.000000,29450.000000,29450.000000


In [11]:
# также смотрим почищенную от пустых значений таблицу клиентов
clients.describe()

,CustomerId,Age
count,8.856000e+03,8856.000000
mean,2.812411e+06,32.814476
std,6.226241e+04,11.360298
min,2.728046e+06,16.000000
25%,2.756260e+06,26.000000
50%,2.795320e+06,31.000000
75%,2.880346e+06,37.000000
max,2.913132e+06,218.000000


In [12]:
# вносим корректировки: если возраст клиента >90 (что в 90+ лет в ресторанах делать???) даем ему средний возраст по таблице
clients['Age'] = clients['Age'].apply(
    lambda x: int(clients['Age'].mean()) if x > 90 else x)

In [13]:
# я сначала даже не подумал об этом, но чтобы учитывать возраст, нужно подменить текущие значения male/female на 0/1
clients['Sex'] = clients['Sex'].astype('category').cat.codes

In [14]:
# смотрим статистику -- поле Sex теперь исчисляемое и может участвовать в работе, а поле Age с адекватными значениями
clients.describe()

,CustomerId,Age,Sex
count,8.856000e+03,8856.000000,8856.000000
mean,2.812411e+06,32.078930,0.420845
std,6.226241e+04,7.976956,0.493723
min,2.728046e+06,16.000000,0.000000
25%,2.756260e+06,26.000000,0.000000
50%,2.795320e+06,31.000000,0.000000
75%,2.880346e+06,36.000000,1.000000
max,2.913132e+06,83.000000,1.000000


In [15]:
# вносим корректировки: если итоговая стоимость после бонусов <0, даем заказу среднюю стоимость по значениям в таблице
orders['SummAfterPointsUsage'] = orders['SummAfterPointsUsage'].apply(
    lambda x: orders['SummAfterPointsUsage'].mean() if x < 0 else x)

In [16]:
# проверяем еще разок заказы -- никаких отрицательных значений
orders.describe()

,CustomerID,Restaurant,Quantity,SummBasic,SummAfterPointsUsage
count,8.816080e+05,881608.000000,881608.000000,881608.000000,881608.000000
mean,2.809192e+06,467.963099,1.154983,287.675306,260.212901
std,6.178279e+04,279.025459,0.650488,264.618404,251.329898
min,2.728046e+06,40.000000,0.000000,0.500000,0.250000
25%,2.754872e+06,325.000000,1.000000,120.000000,110.000000
50%,2.785104e+06,434.000000,1.000000,225.000000,199.000000
75%,2.878847e+06,712.000000,1.000000,379.000000,348.832500
max,2.913132e+06,980.000000,107.000000,29450.000000,29450.000000


In [17]:
# определяем посещал ли клиент ресторан в заданный период
def get_y(df):
    start_date = pd.Timestamp(2017, 7, 1)
    end_date = pd.Timestamp(2017, 12, 31)
    return len(df.loc[(df['RKDate'] >= start_date) & (df['RKDate'] <= end_date)]) > 0

In [18]:
# этот и следующие 2 метода -- вычисление RFM (погуглил). Здесь вычисляем Recency: последняя активность до вычисляемого периода
def get_recency(df):
    start_date = pd.Timestamp(2017, 7, 1)
    orders_before_start = df.loc[(df['RKDate'] < start_date)]
    last_date_before = orders_before_start['RKDate'].max()
    return pd.Timedelta(start_date - last_date_before).days

In [19]:
# здесь Frequency: с какой частотой в месяц клиент проявлял активность
def get_frequency(df):
    start_date = pd.Timestamp(2017, 7, 1)
    first_visit = df[(df['RKDate'] < start_date)].min()
    orders_before_start = len(df.loc[(df['RKDate'] < start_date)])
    if orders_before_start == 0:
        return 0
    months = (pd.Timedelta(start_date - first_visit['RKDate']).days // 30)
    if (months > 0):
        return orders_before_start / months
    else:
        return 0

In [20]:
# здесь Monetary: среднее количество оставленных денег
def get_monetary_value(df):
    start_date = pd.Timestamp(2017, 7, 1)
    orders_before_start = df.loc[(df['RKDate'] < start_date)]
    return orders_before_start['SummAfterPointsUsage'].mean()

In [21]:
# собственный метод: характеристика, оценивающая количество посещенных ресторанов
def get_custom_restaurants(df):
    start_date = pd.Timestamp(2017, 7, 1)
    orders_before_start = df.loc[(df['RKDate'] < start_date)]
    return len(df['Restaurant'].unique())

In [22]:
# итог
def get_client_stats(df):
    stats = [{
        'y': get_y(df),
        'Recency': get_recency(df),
        'Frequency': get_frequency(df),
        'Monetary': get_monetary_value(df),
        'Custom': get_custom_restaurants(df)
    }]
    return pd.DataFrame(data=stats)

# вызов итоговой функции для клиентов
client_stats = orders.groupby(by='CustomerID').apply(
    lambda df: get_client_stats(df))

In [23]:
client_stats.head()

,,y,Recency,Frequency,Monetary,Custom
CustomerID,,,,,,
2728046,0,False,160.0,1.900000,251.789474,6
2728088,0,False,720.0,1.133333,279.647059,2
2728089,0,True,15.0,1.100000,398.734848,2
2728095,0,False,177.0,3.300000,262.737374,2
2728107,0,True,115.0,0.700000,393.714286,3


In [24]:
# слияние таблиц; на всякий случай проводим удаление строк с пустыми значениями
client_stats = client_stats.join(clients[['CustomerId', 'Age', 'Sex']].set_index('CustomerId'), on=['CustomerID'])
client_stats = client_stats.dropna()

In [25]:
client_stats.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8800 entries, (2728046, 0) to (2913132, 0)
Data columns (total 7 columns):
y            8800 non-null bool
Recency      8800 non-null float64
Frequency    8800 non-null float64
Monetary     8800 non-null float64
Custom       8800 non-null int64
Age          8800 non-null float64
Sex          8800 non-null float64
dtypes: bool(1), float64(5), int64(1)
memory usage: 525.2 KB


In [26]:
# выделяем зависимую переменную и характеристики клиентов
y = client_stats['y']
X = client_stats.drop('y', axis=1)

In [27]:
# делим датасет на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=13, test_size=0.2)

In [28]:
# обучаем модель
model = LogisticRegression(random_state=13, solver='lbfgs', max_iter=120)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [29]:
# оценка модели по score
print('Score', model.score(X_test, y_test))

Score 0.7653409090909091


In [30]:
# оценка по precision и recall
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.79      0.73      0.76       889
        True       0.75      0.80      0.77       871

    accuracy                           0.77      1760
   macro avg       0.77      0.77      0.77      1760
weighted avg       0.77      0.77      0.77      1760

